Note this notebook requires that a modified wfsim be installed, and that you have access to a stripped down version of ts_wep. These are not packaged with jf_wep.

In [ ]:
import numpy as np
import galsim
import wfsim
import batoid
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec


In [ ]:
%load_ext autoreload
%autoreload 2

from jf_wep.donutStamp import DonutStamp
from jf_wep.imageMapper import ImageMapper
from jf_wep.wfEstimator import WfEstimator

In [ ]:
from stripped_wep.wfEstimator import WfEstimator as tswepEstimator
from stripped_wep.utility import CamType, DefocalType, getConfigDir

In [ ]:
"""Set of classes for quickly simulating donuts."""
from typing import Optional, Union, no_type_check

import batoid
import galsim
import numpy as np

import wfsim


class Star:
    """A Star object, which contains metadata and images."""

    def __init__(
        self,
        chip: Optional[str] = None,
        intra: Optional[bool] = None,
        angle: Optional[tuple] = None,
        centroid: Optional[tuple] = None,
        flux: Optional[int] = None,
        T: Optional[float] = None,
        band: Optional[str] = None,
        background: Optional[float] = None,
        image: Optional[np.ndarray] = None,
        stamp_size: int = 160,
        opd: Optional[np.ndarray] = None,
        wf_dev: Optional[np.ndarray] = None,
    ) -> None:
        """
        chip: str
            Name of the chip where the star was imaged, in the format "{corner}_{focal}",
            where corner is one of "R00", "R40", "R44", or "R04", and focal is one of
            "SW0" (for extrafocal) or "SW1" (for intrafocal).
        intra: bool
            Whether the star is intrafocal (True) or extrafocal (False).
        angle: tuple
            The field angle of the star in degrees.
        centroid: tuple
            The centroid of the star, in pixels. This is in the global coordinate
            system of the focal plane, not the rotated coordinate systems of the
            individual chips.
        flux: int
            The true flux of the star in photons.
        T: float
            The temperature of the star in K, which was used to determine the
            black body spectrum.
        band: str
            The LSST band the stars are observed in.
        background: float
            Standard deviation of the background noise, in photons.
        image: np.ndarray
            The full image of the chip.
        stamp_size: int
            The side length of the square postage stamp that will be cut out when
            you access the stamp attribute.
        opd: np.ndarray
            Zernike amplitudes in microns, for Noll indices 4-22 (inclusive),
            corresponding to the OPD of the simulated telescope, at the center
            of the CWFS named above.
        wf_dev: np.ndarray
            Zernike amplitudes in microns, for Noll indices 4-22 (inclusive),
            corresponding to the wavefront deviation of the simulated telescope,
            at the center of the CWFS named above.
            The wavefront deviation is the OPD - intrinsic Zernikes.
        """
        self.chip = chip
        self.intra = intra
        self.angle = angle
        self.centroid = centroid
        self.flux = flux
        self.T = T
        self.band = band
        self.background = background
        self.image = image
        self.stamp_size = stamp_size
        self.opd = opd
        self.wf_dev = wf_dev

    @property
    @no_type_check
    def stamp(self) -> np.ndarray:
        w0 = self.stamp_size // 2
        w1 = self.stamp_size - w0
        x, y = self.centroid
        return self.image[y - w0 : y + w1, x - w0 : x + w1].copy()


class Pair:
    """A Pair object, which stores an intra and extrafocal star,
    plus the metadata common to both."""

    def __init__(
        self,
        corner: Optional[str] = None,
        intra: Star = Star(),
        extra: Star = Star(),
    ) -> None:
        """
        Parameters
        ----------
        corner: str
            The corner where the donuts live.
            Can be one of "R00", "R40", "R44", or "R04".
        intra: Star
            The intrafocal star.
        extra: Star
            The extrafocal star.
        """
        self.corner = corner
        self.intra = intra
        self.extra = extra


class DonutSimulator:
    """Object that simulates donuts for the AOS."""

    corner_locations = {
        # bottom left
        "R00": (-0.02075, -0.02075),
        # top left
        "R40": (-0.02075, +0.02075),
        # bottom right
        "R04": (+0.02075, -0.02075),
        # top right
        "R44": (+0.02075, +0.02075),
    }

    def __init__(
        self,
        band: str = "r",
        obs: Optional[dict] = None,
        atm_kwargs: Optional[dict] = None,
        atm: bool = True,
        stamp_size: int = 160,
        seed: int = 42,
    ) -> None:
        """
        Parameters
        ----------
        band: str, default="r"
            The LSST band that the stars are observed in.
        obs: dict, optional
            The observation keyword arguments used by wfsim.
            Any values provided will update default arguments,
            which can be seen in the __init__ method of PairSimulator.
        atm_kwargs: dict, optional
            The atmosphere keyword arguments used by wfsim.
            Any values provided will update default arguments,
            which can be seen in the __init__ method of PairSimulator.
        atm: bool, default=True
            Whether to create an atmosphere for the simulator.
        stamp_size: int, default=160
            Default size of the stamps associated with simulated stars. The stamp
            sizes can be changed ex post facto by changing the corresponding attribute
            on the Star objects.
        seed: int, default=42
            Random seed.
        """
        # load the bandpass and fiducial telescope
        self.band = band
        self.bandpass = galsim.Bandpass(
            f"LSST_{self.band}.dat", wave_type="nm"
        )
        self.telescope0 = batoid.Optic.fromYaml(f"LSST_{self.band}.yaml")
        self.telescope = self.telescope0

        # set some observational parameters
        obs0 = {
            "zenith": 30 * galsim.degrees,
            "raw_seeing": 0.7 * galsim.arcsec,  # zenith 500nm seeing
            "wavelength": self.bandpass.effective_wavelength,
            "exptime": 15.0,  # seconds
            "temperature": 293.0,  # Kelvin
            "pressure": 69.0,  # kPa
            "H2O_pressure": 1.0,  # kPa
        }
        obs = {} if obs is None else obs
        self.obs = obs0 | obs

        # set atmospheric parameters
        atm_kwargs0 = {
            "screen_size": 819.2,
            "screen_scale": 0.1,
            "nproc": 6,  # create screens in parallel using this many CPUs
        }
        atm_kwargs = {} if atm_kwargs is None else atm_kwargs
        self.atm_kwargs = atm_kwargs0 | atm_kwargs

        # set the seed
        self.set_seed(seed)

        # create the simulator
        self.simulator = wfsim.SimpleSimulator(  # type: ignore
            self.obs,
            self.atm_kwargs,
            self.telescope0,
            self.bandpass,
            atm=atm,
            name="R00_SW0",
            rng=self.rng,
        )

        # save the default stamp size
        self.stamp_size = stamp_size

    def set_seed(self, seed: int) -> None:
        """Set the random seed

        Parameters
        ----------
        seed: int
            Random seed.
        """
        self.rng = np.random.default_rng(seed)

    def get_intrinsic_zernikes(self, corner: str) -> np.ndarray:
        """Get the intrinsic Zernikes of the telescope.

        Parameters
        ----------
        corner: str
            The name of the corner to get the OPD for.
            Can be "R00", "R40", "R44", or "R04".

        Returns
        -------
        np.ndarray
            Zernike amplitudes in microns for Noll indices 4-22, inclusive.
        """
        return batoid.zernike(
            self.telescope0,
            *self.corner_locations[corner],
            1e-6,  # reference wavelength = 1 micron
            jmax=22,
            eps=self.telescope0.pupilObscuration,
        )[4:]

    def set_wf_dev(self, wf_dev: np.ndarray) -> None:
        """Set the wavefront deviation, which is the OPD - intrinsic zernikes.

        Parameters
        ----------
        dev: np.ndarray
            Amplitudes of Zernikes in microns, for Noll indices 4-22 (inclusive).
        """
        # create the phase screen
        R_outer = self.telescope0.pupilSize / 2
        R_inner = R_outer * self.telescope0.pupilObscuration
        phase = batoid.Zernike(
            -np.pad(wf_dev, pad_width=(4, 0), mode="constant") * 1e-6,
            R_outer=R_outer,
            R_inner=R_inner,
        )

        # Add phase screen to telescope
        self.telescope = batoid.CompoundOptic(
            (
                batoid.optic.OPDScreen(
                    batoid.Plane(),
                    phase,
                    name="PhaseScreen",
                    obscuration=batoid.ObscNegation(batoid.ObscCircle(5.0)),
                    coordSys=self.telescope0.stopSurface.coordSys,
                ),
                *self.telescope0.items,
            ),
            name="PerturbedLSST",
            backDist=self.telescope0.backDist,
            pupilSize=self.telescope0.pupilSize,
            inMedium=self.telescope0.inMedium,
            stopSurface=self.telescope0.stopSurface,
            sphereRadius=self.telescope0.sphereRadius,
            pupilObscuration=self.telescope0.pupilObscuration,
        )

    def set_opd(self, opd: np.ndarray, corner: str) -> None:
        """Directly set the OPD of the telescope.

        Parameters
        ----------
        opd: np.ndarray
            Amplitudes of Zernikes in microns, for Noll
            indices 4-22 (inclusive).
        corner: str
            The name of the corner to get the OPD for.
            Can be "R00", "R40", "R44", or "R04".
        """
        wf_dev = opd - self.get_intrinsic_zernikes(corner)
        self.set_wf_dev(wf_dev)

    def set_dof(self, dof: np.ndarray) -> None:
        """Perturb the degrees of freedom of the telescope

        Parameters
        ----------
        dof: np.ndarray
            Perturbations to degrees of freedom of the telescope.
            Contains 50 entries, with definitions and units listed below:
            - 0: M2 dz (microns)
            - 1: M2 dx (microns)
            - 2: M2 dy (microns)
            - 3: M2 x-axis rotation (arcsecs)
            - 4: M2 y-axis rotation (arcsecs)
            - 5: Camera dz (microns)
            - 6: Camera dx (microns)
            - 7: Camera dy (microns)
            - 8: Camera x-axis rotation (arcsecs)
            - 9: Camera y-axis rotation (arcsecs)
            - 10-29: M1M3 bending modes (microns)
            - 30-49: M2 bending modes (microns)

        """
        # create perturbed telescope from DOF
        self.telescope = wfsim.SSTFactory(self.telescope0).get_telescope(dof=dof)  # type: ignore

    def get_opd(self, corner: str) -> np.ndarray:
        """Get the OPD of the telescope at the center of the given corner CWFS.

        Parameters
        ----------
        corner: str
            The name of the corner to get the OPD for.
            Can be "R00", "R40", "R44", or "R04".

        Returns
        -------
        np.ndarray
            Zernike amplitudes in microns for Noll indices 4-22, inclusive.
        """
        # get the OPD from batoid
        opd = batoid.zernike(
            self.telescope,
            *self.corner_locations[corner],
            1e-6,  # reference wavelength = 1 micron
            jmax=22,
            eps=self.telescope.pupilObscuration,
        )[4:]

        return opd

    def get_wf_dev(self, corner: str = "R00") -> np.ndarray:
        """Get the wavefront deviation at the center of the corner CWFS.

        The wavefront deviation is the OPD - intrinsic Zernikes.

        Parameters
        ----------
        corner: str
            The name of the corner to get the wavefront deviation for.
            Can be "R00", "R40", "R44", or "R04".

        Returns
        -------
        np.ndarray
            Zernike amplitudes in microns for Noll indices 4-22, inclusive.
        """
        wf_dev = self.get_opd(corner) - self.get_intrinsic_zernikes(corner)

        return wf_dev

    def simulate_star(
        self,
        angle: Union[list, str, None] = "center",
        flux: int = 1_000_000,
        T: float = 8_000,
        background: float = 10,
        chip: str = "R00_SW1",
        intra: bool = True,
    ) -> Star:
        """Simulate a donut pair.

        Parameters
        ----------
        angle: list, optional
            List of [field_x, field_y] angles, in degrees, for
            the star. If "center", the center of the chip is used.
            If None, random angle drawn.
        flux: int, default=1_000_000
            Number of photons for the star.
        T: float, default=8_000
            Temperature, in Kelvin, which is used to generate a
            blackbody spectrum for the star.
        background: float, default=10
            Standard deviation of the background noise, in photons.
        chip: str, default="R00_SW1"
            The chip on which to simulate the star.
        intra: bool, default=True
            Whether to simulate an intrafocal (True) or extrafocal (False) donut.

        Returns
        -------
        Star
            A star object with metadata and an image.
        """
        # create the star
        star = Star()

        # determine the focal offset
        star.intra = intra
        offset = -1.5e-3 if star.intra else +1.5e-3
        self.simulator.telescope = self.telescope.withGloballyShiftedOptic(
            "Detector", [0, 0, offset]
        )

        # set the CCD to simulate
        star.chip = chip
        self.simulator.set_name(chip)

        # determine the field angle
        if angle == "center":
            star.angle = np.rad2deg(self.simulator.get_bounds().mean(axis=1))
        elif angle is None:
            # randomly sample centroid
            buffer = np.ceil(self.stamp_size / 2).astype(int)
            bounds = self.simulator.image.bounds.withBorder(-buffer)
            x = self.rng.uniform(bounds.xmin, bounds.xmax)
            y = self.rng.uniform(bounds.ymin, bounds.ymax)

            # convert centroid to field angle
            star.angle = np.array(self.simulator.wcs.xyToradec(x, y, galsim.degrees))  # type: ignore
        else:
            star.angle = np.array(angle)  # type: ignore

        # save the donut centroid
        x, y = self.simulator.wcs.radecToxy(*star.angle, galsim.degrees)
        x = int(x - self.simulator.image.bounds.xmin)  # x in image coords
        y = int(y - self.simulator.image.bounds.ymin)  # y in image coords
        star.centroid = np.array([x, y])  # type: ignore

        # save the star properties
        star.flux = int(flux)
        star.T = float(T)
        sed = wfsim.BBSED(star.T)  # type: ignore

        # and the band we observe in
        star.band = self.band

        # calculate pixel variance of background
        star.background = float(background)
        variance = star.background**2

        # simulate the star
        self.simulator.add_star(*np.deg2rad(star.angle), sed, star.flux, self.rng)  # type: ignore
        self.simulator.add_background(variance, self.rng)

        # save the full image
        star.image = self.simulator.image.array.copy()

        # and the default stamp size
        star.stamp_size = self.stamp_size

        # finally save the wavefront from the simulation
        # star.opd = self.get_opd(corner)
        # star.wf_dev = self.get_wf_dev(corner)

        return star

    def simulate_pair(
        self,
        intra_angle: Union[list, str, None] = "center",
        extra_angle: Union[list, str, None] = "center",
        intra_flux: int = 1_000_000,
        extra_flux: int = 1_000_000,
        intra_T: float = 8_000,
        extra_T: float = 8_000,
        background: float = 10,
        corner: str = "R00",
        chip: Optional[str] = None,
    ) -> Pair:
        """Simulate a donut pair.

        Parameters
        ----------
        intra_angle: list, optional
            List of [field_x, field_y] angles, in degrees, for
            the intrafocal donut. If "center", the center of
            the chip is used. If None, random angle drawn.
        extra_angle: list, optional
            List of [field_x, field_y] angles, in degrees, for
            the extrafocal donut. If "center", the center of
            the chip is used. If None, random angle drawn.
        intra_flux: int, default=1_000_000
            Number of photons for intrafocal donut.
        extra_flux: int, default=1_000_000
            Number of photons for extrafocal donut.
        intra_T: float, default=8_000
            Temperature, in Kelvin, which is used to generate a
            blackbody spectrum for the intrafocal donut.
        extra_T: float, default=8_000
            Temperature, in Kelvin, which is used to generate a
            blackbody spectrum for the extrafocal donut.
        background: float, default=10
            Standard deviation of the background noise, in photons.
        corner: str, default="R00"
            The corner for which the donuts are simulated. Can be
            one of "R00", "R40", "R44", or "R04". If supplied, the
            intra and extrafocal stars are automatically placed on
            the correct chips. This parameter is only used if chip
            is None.
        chip: str, default=None
            The chip on which to simulate the stars.

        Returns
        -------
        Pair
            A pair object with metadata, and an intra and
            extrafocal donut.
        """
        # Determine the chip names
        if chip is None:
            intra_chip = f"{corner}_SW1"
            extra_chip = f"{corner}_SW0"
        else:
            intra_chip = extra_chip = chip

        # create the pair
        pair = Pair()

        # simulate the intrafocal star
        pair.intra = self.simulate_star(
            angle=intra_angle,
            flux=intra_flux,
            T=intra_T,
            background=float(background),
            chip=intra_chip,
            intra=True,
        )

        # simulate the extrafocal star
        pair.extra = self.simulate_star(
            angle=extra_angle,
            flux=extra_flux,
            T=extra_T,
            background=float(background),
            chip=extra_chip,
            intra=False,
        )

        return pair


In [ ]:
def plotZernikeEstimationPoster(
    intraAngle,
    extraAngle,
    chip,
    amp=5 * [500e-9] + 5 * [200e-9] + 6 * [100e-9] + 2 * [50e-9],
    jmax=22,
    jmaxEst=22,
    background=0,
    batoid=True,
    model="offAxis",
    band="r",
):
    # Create the wavefront estimator
    wfEst = WfEstimator(
        jmax=jmaxEst,
        algoConfig=dict(
            saveHistory=True, maxIter=30, compGain=0.7, opticalModel=model
        ),
    )

    # Create the figure and gridspec
    nPanels = jmax - 3
    fig = plt.figure(figsize=(11, 4.2 * nPanels))
    gs = GridSpec(
        2 * nPanels,
        1,
        figure=fig,
        height_ratios=nPanels * [1, 0.01],
    )

    if isinstance(amp, float):
        amp = [amp] * nPanels
    ampIter = iter(amp)

    for j in range(4, jmax + 1):
        # Calculate the 0-indexed counter
        i = j - 4

        # Perturb the jth Zernike coefficient
        zkTrue = np.zeros(jmaxEst - 3)
        zkTrue[i] = next(ampIter, amp[-1])

        # Simulate the pair of stars
        if batoid:
            # Simulate the stars with batoid
            sim = DonutSimulator(atm=False, stamp_size=180)
            sim.set_wf_dev(zkTrue * 1e6)
            pair = sim.simulate_pair(
                intra_angle=intraAngle,
                extra_angle=extraAngle,
                background=background,
                chip=chip,
            )

            # Package the stars in DonutStamps
            intra = DonutStamp(
                pair.intra.stamp, pair.intra.angle, "intra", band
            )
            extra = DonutStamp(
                pair.extra.stamp, pair.extra.angle, "extra", band
            )
        else:
            # Simulate the stars using the image mapper model
            mapper = ImageMapper(opticalModel=model)

            intra = mapper.mapPupilToImage(
                DonutStamp(
                    np.zeros((180, 180)),
                    intraAngle,
                    "intra",
                    band,
                ),
                zkTrue,
            )
            intra.image *= 1e6 / intra.image.sum()

            extra = mapper.mapPupilToImage(
                DonutStamp(
                    np.zeros((180, 180)),
                    extraAngle,
                    "extra",
                    band,
                ),
                zkTrue,
            )
            extra.image *= 1e6 / extra.image.sum()

            # Add Poisson noise
            rng = np.random.default_rng(42)
            intra.image += rng.normal(scale=np.sqrt(intra.image))
            extra.image += rng.normal(scale=np.sqrt(extra.image))

            # Add background noise
            intra.image += rng.normal(scale=background, size=intra.image.shape)
            extra.image += rng.normal(scale=background, size=extra.image.shape)

        # Estimate the wavefront using jf_wep
        zk = wfEst.estimateWf(intra, extra)

        # Pull the final iteration out of the history
        finalIter = max(wfEst.algo.history)
        history = wfEst.algo.history[finalIter]

        # Also estimate using ts_wep
        tswep = tswepEstimator(f"{getConfigDir()}/cwfs/algo")
        tswep.config(sizeInPix=intra.image.shape[0], opticalModel=model)
        tswep.setImg(intra.fieldAngle, DefocalType.Intra, image=intra.image)
        tswep.setImg(extra.fieldAngle, DefocalType.Extra, image=extra.image)
        zkTSWEP = tswep.calWfsErr() * 1e-9

        # ts_wep predicts the full OPD when in onAxis mode
        # therefore we need to subtract the intrinsic Zernikes for fair comparison
        if model == "onAxis":
            zkTSWEP -= wfEst.instrument.getIntrinsicZernikes(
                *intra.fieldAngle, band, jmax=22
            )

        # Create a gridspec for the subpanel
        gsi = GridSpecFromSubplotSpec(
            2,
            5,
            width_ratios=[0.5, 1, 1, 0.3, 3],
            subplot_spec=gs[2 * i],
        )

        # Add info to the first subplot
        ax0 = fig.add_subplot(gsi[:, 0])
        ax0.axis("off")
        names = {
            4: "Defocus",
            5: "Primary\nAstigmatism\n(oblique)",
            6: "Primary\nAstigmatism\n(vertical)",
            7: "Primary\nComa\n(vertical)",
            8: "Primary\nComa\n(horizontal)",
            9: "Trefoil\n(vertical)",
            10: "Trefoil\n(oblique)",
            11: "Primary\nSpherical",
            12: "Secondary\nAstigmatism\n(vertical)",
            13: "Secondary\nAstigmatism\n(oblique)",
            14: "Quadrafoil\n(vertical)",
            15: "Quadrafoil\n(oblique)",
            16: "Secondary\nComa\n(horizontal)",
            17: "Secondary\nComa\n(vertical)",
            18: "Secondary\nTrefoil\n(oblique)",
            19: "Secondary\nTrefoil\n(vertical)",
            20: "Pentafoil\n(oblique)",
            21: "Pentafoil\n(vertical)",
            22: "Secondary\nSpherical",
            23: "Tertiary\nAstigmatism\n(oblique)",
            24: "Tertiary\nAstigmatism\n(vertical)",
            25: "Secondary\nQuadrafoil\n(oblique)",
            26: "Secondary\nQuadrafoil\n(vertical)",
            27: "Hexafoil\n(oblique)",
            28: "Hexafoil\n(vertical)",
        }
        ax0.text(
            0.5,
            0.5,
            (
                f"j = {j}"
                "\n\n"
                f"{names[j]}"
                "\n\n\n"
                f"N iters: {finalIter}"
                "\n\n"
                f"Converged?\n{history['converged']}"
                "\n\n"
                f"Caustic?\n{history['caustic']}"
            ),
            ha="center",
            va="center",
            transform=ax0.transAxes,
            fontweight="bold",
        )

        # Plot the images in the first column
        ax1 = fig.add_subplot(gsi[0, 1])
        ax2 = fig.add_subplot(gsi[1, 1])
        ax1.set(title="Intra", xticks=[], yticks=[])
        ax2.set(title="Extra", xticks=[], yticks=[])
        ax1.imshow(history["intraCent"], origin="lower")
        ax2.imshow(history["extraCent"], origin="lower")

        # Plot I0 and dI/dz in the second column
        ax3 = fig.add_subplot(gsi[0, 2])
        ax4 = fig.add_subplot(gsi[1, 2])
        ax3.set(title="I0", xticks=[], yticks=[])
        ax4.set(title="dI/dz", xticks=[], yticks=[])
        ax3.imshow(history["I0"], origin="lower")
        ax4.imshow(history["dIdz"], origin="lower")

        # Plot the Zernike amplitudes on the top right
        ax5 = fig.add_subplot(gsi[0, 4])
        ax5.set(
            ylabel="Amplitude (nm)",
            xticks=np.arange(4, 23, 2),
            xlim=(3.5, 22.5),
        )
        ax5.plot(np.arange(4, len(zk) + 4), zk * 1e9, label="TIE", c="C1")
        ax5.plot(
            np.arange(4, len(zkTSWEP) + 4),
            zkTSWEP * 1e9,
            label="tswep",
            c="C2",
        )
        ax5.plot(
            np.arange(4, len(zkTrue) + 4),
            zkTrue * 1e9,
            label="Truth",
            c="k",
            ls="--",
        )
        ax5.legend(
            bbox_to_anchor=(0, 1.02, 1, 0.2),
            loc="lower center",
            borderaxespad=0,
            ncol=4,
            fontsize=10,
        )

        # Plot the Zernike residuals in the bottom right
        ax6 = fig.add_subplot(gsi[1, 4])
        ax6.set(
            ylabel="Residual (nm)",
            xticks=np.arange(4, 23, 2),
            xlim=(3.5, 22.5),
        )
        ax6.plot(
            np.arange(4, len(zk) + 4), (zk - zkTrue) * 1e9, c="C1", alpha=0.5
        )
        ax6.plot(
            np.arange(4, len(zkTSWEP) + 4),
            (zkTSWEP - zkTrue) * 1e9,
            c="C2",
            alpha=0.5,
        )
        ax6.plot(
            np.arange(4, len(zkTrue) + 4),
            0 * zkTrue * 1e9,
            label="Truth",
            c="k",
            ls="--",
        )
        ax6.set(
            xlabel="Noll index",
            ylabel="Residual (nm)",
            xticks=np.arange(4, len(zk) + 4, 2),
        )

        # Add a line below the plot
        gsi = GridSpecFromSubplotSpec(1, 1, subplot_spec=gs[2 * i + 1])
        ax = fig.add_subplot(gsi[0])
        ax.axhline(-3, c="k", clip_on=False)
        ax.set(ylim=(0, 1))
        ax.axis(False)

    plt.subplots_adjust(hspace=0.3)


# Near center, onAxis model

In [ ]:
plotZernikeEstimationPoster(
    intraAngle=(0.1, -0.05),
    extraAngle=(0.1, -0.05),
    chip="R22_S11",
    batoid=True,
    model="onAxis",
)

# Near center, offAxis model

In [ ]:
plotZernikeEstimationPoster(
    intraAngle=(0.1, -0.05),
    extraAngle=(0.1, -0.05),
    chip="R22_S11",
    batoid=True,
    model="offAxis",
)

# In corner, offAxis model, same angles

In [ ]:
plotZernikeEstimationPoster(
    intraAngle=(-1.189, -1.125),
    extraAngle=(-1.189, -1.125),
    chip="R00_SW0",
    batoid=True,
    model="offAxis",
)

# In corner, offAxis model, different angles

In [ ]:
plotZernikeEstimationPoster(
    intraAngle=(-1.189, -1.125),
    extraAngle=(-1.212, -1.125),
    chip="R00_SW0",
    batoid=True,
    model="offAxis",
)